In [54]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [55]:
os.environ["GROQ_API_KEY"] = "your_api_key"

In [56]:
llm = ChatGroq( groq_api_key = os.environ["GROQ_API_KEY"],temperature = 0.6,model="Gemma2-9b-it")

In [57]:
loader=UnstructuredURLLoader(urls=
                          ["https://www.rsm.global/insights/automotive-trends-2025-impacting-middle-market-organisations",
                            "https://www.dentons.com/en/insights/articles/2025/january/23/trends-and-challenges-shaping-the-automotive-industry-in-2025"])
data = loader.load()
len(data)

2

In [58]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
docs = text_splitter.split_documents(data)

In [59]:
len(docs)

34

In [60]:
docs[3]

Document(metadata={'source': 'https://www.rsm.global/insights/automotive-trends-2025-impacting-middle-market-organisations'}, page_content='Electric vehicle trends in 2025\n\nStill not the EV revolution once thought but still plenty of growth\n\nFigures from the last few years show that the EV adoption rate is still slow at a global level. However, the share of cars that are electric (hybrid and full electric) continues to rise. Also, despite the slowing adoption rate, data has shown that the number of electric and hybrid vehicles actually on the road has dramatically increased (from 26 million in 2022 to 40 million in 2023). The EV market is expected to continue growing well into the 2030s. Regardless, with the continued global slowdown of EV adoption, we can also expect original equipment manufacturers (OEMs) to follow suit, with a decrease in the level of production, especially in the West.')

In [61]:
for doc in docs:
    print(len(doc.page_content))

979
494
994
762
511
705
773
417
754
867
901
905
795
979
855
870
983
721
493
530
688
740
657
810
649
992
778
594
949
869
729
594
994
764


In [62]:
embedding = HuggingFaceEmbeddings()
vectorindex_hugging = FAISS.from_documents(docs,embedding)

C:\Users\apasi\AppData\Local\Temp\ipykernel_13904\1919704768.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


In [63]:
file_path = "vector_index.pkl"
with open(file_path,"wb") as file:
    pickle.dump(vectorindex_hugging, file)

In [64]:
if os.path.exists(file_path):
    with open(file_path,"rb") as f:
        VectorIndex = pickle.load(f)
    

In [65]:
chain=RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=VectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F11A964E20>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F118ED6890>, model_name='Gemma2-9b-it', temperature=0.6, model_kwargs={}, groq_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'su

In [66]:
query = " What is Impact of US tariff policies ?"
langchain.debug = True
chain({"question":query},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": " What is Impact of US tariff policies ?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The recently proposed 25% tariffs on imports from Mexico and Canada, while temporarily delayed, may also have a drastic impact on the global automotive supply chain. With many big names, such as Volkswagen, BMW, and Ford, having manufacturing plants in Mexico and both countries seeing around 90% of their auto exports go to the U.S., these tariffs could have a huge knock-on effect on global automotive trade dynamics throughout the value chain in 2025.\n\n“The new U.S. administration is keen to drive manufacturing in all its forms back to American shores, und

{'answer': 'The proposed 25% tariffs on imports from Mexico and Canada may have a drastic impact on the global automotive supply chain. With many big names, such as Volkswagen, BMW, and Ford, having manufacturing plants in Mexico and both countries seeing around 90% of their auto exports go to the U.S., these tariffs could have a huge knock-on effect on global automotive trade dynamics throughout the value chain in 2025.  The new U.S. administration is keen to drive manufacturing in all its forms back to American shores, underscored by both the forthcoming 25% tariffs on imported steel, aluminium, and their derivatives. This is in addition to the most recent messaging that imported autos (along with semiconductors) will also be hit with 25% tariffs. Notably, this is all on top of the latest additional 10% tariffs on Chinese products as well as the potentially upcoming reciprocal tariffs on all countries.\n',
 'sources': 'https://www.rsm.global/insights/automotive-trends-2025-impacting-